### 特征提取：
1. 单个传感器特征
2. 传感器对特征
3. 时间传感器特征
4. 时间特征

In [2]:
import pandas as pd
import utils
import numpy as np


# sensorType = ["D0","L0","LS","M0","MA","T0"]
# 不考虑连续型传感器

sensorType = ["D0","L0","M0","MA"]
def loadSensor(file):
    sensors = []
    with open(file,'r') as sf:
        for line in sf:
            sensors.append(line.strip())
    return sensors

#  加载传感器ID
continueSensors = loadSensor("./data/continue_sensors.txt")

binarySensors = loadSensor("./data/bin_sensors.txt")



def dataframeActivity(activity):
    """
    输入：一个完整的观测序列
    输出：dataframe类型的数据结构
    
    """
    df_act = pd.DataFrame(activity[1:],columns=['date','time','sensor','value'])
    
    return df_act
    
    

    

#### 单个传感器类型特征 ：不考虑连续型传感器
1. 每个传感器出现的次数
2. 第一个出现的传感器
3. 最后一个出现的传感器
4. 共出现了几种传感器
5. 每类传感器出现的次数["D0","L0","M0","MA"]

In [24]:
def onesensorFeatures(df_act):
#     每种传感器出现的次数
    sensors_num = np.zeros(len(binarySensors))
    for sid in binarySensors:
        count_sensor = df_act.sensor == sid
        sensors_num[binarySensors.index(sid)] = count_sensor.sum()
#     print "sensors_num:",sensors_num
# 第一个传感器
    i = 0 
    while i < df_act.shape[0]:
        start_sensor = df_act.iloc[i][2]
        if start_sensor in binarySensors:
            break
        i += 1
# 最后一个传感器
    j = df_act.shape[0]
    while j > 0:
        finish_sensor = df_act.iloc[i][2]
        if finish_sensor in binarySensors:
            break
        j -= 1    
#     print "start_sensor,finish_sensor:",start_sensor,finish_sensor
    ss = np.zeros(len(binarySensors))
    fs = np.zeros(len(binarySensors))
    ss[binarySensors.index(start_sensor)] = 1
    fs[binarySensors.index(finish_sensor)] = 1
    
# 传感器种类数:二值传感器
    sensors = set(df_act.sensor)
    num_sensor_type = len(sensors.intersection(binarySensors))
#     print "num_sensor_type",num_sensor_type
    
# 各类传感器出现的次数 sensorType = ["D0","L0","M0","MA"]
    cat_num = np.zeros(len(sensorType))
    for i in range(df_act.sensor.shape[0]):
        s = df_act.iloc[i][2]
        if s in binarySensors:
            cat_num[sensorType.index(s[0:2])] += 1
    
    sf = np.hstack((sensors_num,ss,fs,num_sensor_type,cat_num))
    
    return sf

    
    





In [ ]:
def sensorpairFeatures(df_act):
    pass
    



#### 时间传感器特征：不同时间段是否出现

1. 前三分之一时间段内出现传感器
2. 中间三分之一时间段出现的传感器
3. 最后三分之一时间段出现的传感器

In [ ]:

def sensorTimeFeatures(df_act):
    pass

### 时间类型的特征

In [4]:

def timeFeatures(df_act):
    """
    返回时间相关特征
    时间间隔（delta，分钟）、是否跨天（is_two_day）,是否是工作日？周六？周日？开始时间,结束时间（小时）
    """
    from datetime import datetime,timedelta
    ts = np.array(df_act.time)
    ds = np.array(df_act.date)
    
    sy,sm,sd = ds[0].split("-")
    sh,sminutes = ts[0].split(":")[0:-1]
    ss,sns = ts[0].split(":")[-1].split(".")
    
    ey,em,ed = ds[-1].split("-")
    eh,eminutes = ts[-1].split(":")[0:-1]
    es,ens = ts[-1].split(":")[-1].split(".")
    
    starttime = datetime(int(sy),int(sm),int(sd),int(sh),int(sminutes),int(ss),int(sns))
    endtime = datetime(int(ey),int(em),int(ed),int(eh),int(eminutes),int(es),int(ens))
    
    
    delta = (endtime - starttime).total_seconds()
    delta = int(delta)


#     是否跨天
    is_two_day = 0
    if ds[0] != ds[-1]:
        is_two_day = 1
    
#     是否是周六？周日？工作日？
    day_of_week = pd.DatetimeIndex(df_act.date).dayofweek[-1] + 1
    is_workingday = 1
    is_sat = 0
    is_sun = 0
    if day_of_week == 6:
        is_workingday = 0
        is_sat = 1
    if day_of_week == 7:
        is_workingday = 0
        is_sun = 1
    
#     开始时间
    start_hour = int(sh)
#     结束时间
    end_hour = int(eh)

    return np.array((delta,is_two_day,is_workingday,is_sat,is_sun,start_hour,end_hour)).flatten()

In [26]:
filename = "./data/ann.txt"
activities = utils.ExtractActivities(filename)

tfs = []
sfs = []
spfs = []
stfs = []

df = dataframeActivity(activities[0])
tf = timeFeatures(df)
sf = onesensorFeatures(df)

print tf,sf

# for act in activities:
#     df = dataframeActivity(act)
#     tf = timeFeatures(df)
#     tfs.append(tf)
#     sf = onesensorFeatures(df)
#     sfs.append(sf)
# df_time_features = pd.DataFrame(tfs,columns=["length",'istwoday',"isworkingday","issat","issun","starthour","endhour"])

# print df_time_features.head
# print sfs[0]



[12331     0     1     0     0     2     5] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  0.  0.  2.  0.]
